# Coding Fastspeech2 a TTS system
> Part of a series where I read and implement deep learning papers

- toc: true
- badges: true
- comments: true
- author: Jesse Deng
- image: images/mel.jpeg
- categories: [fastpages, jupyter]

# What I learnt summary

- `Isolate various modules if loss not converging, check inputs and outputs to each section. Ensure data is correct.`
Most important thing I learnt, check all inputs and outputs after setting everything up, especially if you think its all been done correctly. 
I had normalised the pitch data incorrectly to begin with, this caused the model to appear to converge however on the testset only garbage was being produced. Took me a while but I found the error by looking at the raw numerical data and realizing most of the values where centered around the number -4, I went back to the datapipeline and realized I had incorrectly normalized the values... SMH.

- `Overfit on a subset of the data say, 10 datapoints.` This is useful as if your model is really broken even this won't converge, however can be decieving as even with a incorrectly normalized feature-set (pitch) the model was converging. This led me to believe that everything was okay when it really wasn't, (Got this from Andrej Karpathys lectures on Youtube).

- `Start with the simplest working prototype and build up` Obvious, but my excitement at working on a new project caused me to add additional complexity before the core featureset was in place.


- `Where possible extract additional features from the data and train preceding sections of the model` to facilitate faster convergance and direct the model search space towards the final objective. Fastspeech2 uses pitch and energy feature embeddings, training sections of the network preceding the mel output.

- `Identical inputs that map to different output distributions require additional information`
I began by training on a multispeaker dataset (LibriSpeech) and the model was creating poor outputs, In hind sight this was obviously due to the fact that the same words by different speakers can result in rather disparate outputs (eg: male voice vs female voice with words in different contexts). To improve the model performance speaker embeddings where required.
`TLDR; Inform the model of input/output idiosyncrasies by adding additional information.`

- `Residuals are great.` Adding a residual improved the loss by a large factor, (due to increasing gradient flow).

- `Regularization is super important`
 dropout, batchnorm etc. 


# Data-pipline

The most important thing to a deep learning model is the data, Fastspeech2 (FS2) is clever in that it extracts additional features (pitch and energy) from the audio and essentially trains various parts of the model to learn the relationship between input phonemes and those features. 

Step 1 was to create an efficient data pipeline, FS2 uses phonemes, pitch, energy and phoneme durations as inputs whilst training and Melspectrograms https://en.wikipedia.org/wiki/Mel-frequency_cepstrum pitch, energy and durations as targets to differentiate against.
All data was normalized by subtracting the mean and dividing by the standard deviation (min max normalization will be tested in the future).

I also wanted the pipeline to be as modular as possible for any subsequent dataset, so that anyone who wishes to use the model with their own dataset can.

My design was to have an initial preprocessing function that users would write themselves and is unique to the dataset, once in the correct format call one command in the root directory and it will align and create the dataset from there with the data saved as numpy files.

`Messy data -> Standardize to same format -> generate dataset -> input into model`

# Model

### Quantized Embeddings

Pitch and energy predictions where quantized, so the pitch / energy predictor takes as input the phoneme embedding and predicts the feature vector, the values are then rounded to the closest vector in the feature bins. This restricts the possible search space down to a limited range, however a downside is that the desired features may be continuous in vector space this can be addressed by creating a larger number of bins. Once the predictions have been quantized an embedding layer is used to project the predictions into a descriptive vector space in a similar way to how word embeddings work in NLP.

### Duration Predictor & Length Regulator

Duration predictor predicts Log values so it dosn't have to output crazy high numbers just remember to set your training target to the same value. 
First time I coded up the length regulator I was using too many slow for loops, Once I got the quick and dirty version up and running I came back and vectorized as many parts as I could.

### Transformer

Last year I spent several days reading blog posts and watching videos about transformer architechture from the canonical work 'Attention is all you need' https://arxiv.org/abs/1706.03762 The idea of self attention is relativly simple to implement but extremely effective across domains.
One particular nuance that took me a while to clock was the query, key and values are projected to a larger vector space and then rearranged, splitting the vector into so called 'heads', this allows for the self attention mechanism to be implemented in parallel allowing for quicker processing speed when compared to auto regressive methods such as LSTM's.


# Future work

This is the first blog I'm doing on programming projects im working on. 
Its to serve as a refrence when so I don't repeat ridiculous mistakes as well as document anything intresting I'v learnt and found.. Ill be adding this to my mobile app soon which will also be open sourced and available on the google and app store.

Ill also be using fastspeech2 to implement a speech cloning software.. check back for more :)

Thanks for reading

Original paper https://arxiv.org/abs/2006.04558